In [ ]:
#| default_exp core

In [ ]:
#| hide
#| export

from __future__ import annotations
import duckdb
from duckdb import DuckDBPyConnection
from duckdb import DuckDBPyRelation
from typing import List, Dict, Optional, Union, Any, Tuple, Set
from fastcore.all import store_attr, patch, L
import numpy as np
import pandas as pd
from dataclasses import field, make_dataclass
from fastcore.xtras import hl_md, dataclass_src
from functools import wraps, partial

### Wrapping DuckDB Python API

When you `import duckdb`, there are two main concepts that you have to handle:


A *DuckDBPyConnection* represents a connection to a DuckDB database in a file or memory

In [ ]:
db = duckdb.connect('../data/chinook.duckdb')
db

<duckdb.duckdb.DuckDBPyConnection>

DuckDB API has no concept of a Database, only of a connection that may envolve different attached databases (or catalogs) and schemas.

A Catalog is just a holder for schemas, and schemas hold catalog entries, like tables, views, functions, types, etc..

We will change this behaviour in a bit...

A DuckDBPyRelation represents a query. It is a table-like object that will be lazy executed and can be queried further. 

Once it's executed then yes, it contains the result set.

But when further projections are made on it, that result set is not used, the projections are just done on top of the original query as a subquery

`duckdb.table('tbl').sql("select a")`


Really becomes, in essence:
`select a from (select * from tbl)`


In [ ]:
db.sql(f"FROM duckdb_tables()").select('table_name')

┌───────────────┐
│  table_name   │
│    varchar    │
├───────────────┤
│ Album         │
│ Artist        │
│ Customer      │
│ Employee      │
│ Genre         │
│ Invoice       │
│ InvoiceLine   │
│ latin         │
│ latin1view    │
│ MediaType     │
│ Playlist      │
│ PlaylistTrack │
│ todos         │
│ Track         │
├───────────────┤
│    14 rows    │
└───────────────┘

In [ ]:
type(db.sql(f"SELECT table_name FROM duckdb_tables()"))

duckdb.duckdb.DuckDBPyRelation

### Improving Connection

We want the developer to understand the connection as a database.

```python

import duckdb

conn = duckdb.connect()

```

becomes


```python

from fastduck import duckdb

db = duckdb.connect()

```
By acessing the same Python API using `fastduck`, the developer shall get some niceties.



Let's start by simplifying the access to some information on the current catalog and schema in the connection.

In [ ]:
#| export

def _current(self: DuckDBPyConnection): return self.sql('select current_catalog, current_schema').fetchone()
@patch(as_prop=True)
def catalog(self: DuckDBPyConnection): return _current(self)[0]

@patch(as_prop=True)
def schema(self: DuckDBPyConnection): return _current(self)[1]

@patch(as_prop=True) # alias for alias
def name(self:DuckDBPyRelation): return self.alias

@patch
def __getitem__(self:DuckDBPyRelation, idxs) -> DuckDBPyRelation: # selecting by passing a list of column names
    return self.select(*idxs) if isinstance(idxs, Union[List, Set, Tuple]) else self.select(idxs)
@patch 
def to_recs(self:DuckDBPyRelation) -> List[Dict[str, Any]]:
    '''The relation as a list of records'''
    return self.df().to_dict(orient='records')

@patch 
def q(self:DuckDBPyConnection, *args, **kwargs) -> List[Dict[str, Any]]:
    '''Run a query and return the result as a list of records'''
    return self.sql(*args, **kwargs).to_recs()



In [ ]:
album = db.sql("select * from Album")
album['AlbumId', 'Title'].limit(5)

┌─────────┬───────────────────────────────────────┐
│ AlbumId │                 Title                 │
│  int32  │                varchar                │
├─────────┼───────────────────────────────────────┤
│       1 │ For Those About To Rock We Salute You │
│       2 │ Balls to the Wall                     │
│       3 │ Restless and Wild                     │
│       4 │ Let There Be Rock                     │
│       5 │ Big Ones                              │
└─────────┴───────────────────────────────────────┘

In [ ]:
db.table('Album')['AlbumId', 'Title'].limit(2).to_recs()

[{'AlbumId': 1, 'Title': 'For Those About To Rock We Salute You'},
 {'AlbumId': 2, 'Title': 'Balls to the Wall'}]

We also need a way to know the tables in the database.


In [ ]:
#| export

@patch
def _tables(self: DuckDBPyConnection, catalog:str=None) -> DuckDBPyRelation:
    '''Returns a dictionary of tables in the database'''
    q = f"from {catalog or self.catalog}.information_schema.tables"
    s = f"'{catalog or self.catalog}' as catalog, table_schema as schema, table_name as name, table_type as type, table_comment as comment"
    return self.sql(q).distinct().select(s)
@patch(as_prop=True)
def tables(self: DuckDBPyConnection) -> DuckDBPyRelation: return self._tables()

In [ ]:
@patch(as_prop=True)
def views(self: DuckDBPyConnection) -> DuckDBPyRelation:
    '''Returns a dictionary of views in the database'''
    return self.tables.filter("type =='VIEW'")
@patch(as_prop=True)
def base_tables(self: DuckDBPyConnection) -> DuckDBPyRelation:
    '''Returns a dictionary of views in the database'''
    return self.tables.filter("type =='BASE TABLE'")

In [ ]:
db.base_tables

┌─────────┬─────────┬───────────────┬────────────┬─────────────┐
│ catalog │ schema  │     name      │    type    │   comment   │
│ varchar │ varchar │    varchar    │  varchar   │   varchar   │
├─────────┼─────────┼───────────────┼────────────┼─────────────┤
│ chinook │ main    │ Album         │ BASE TABLE │ Album table │
│ chinook │ main    │ Artist        │ BASE TABLE │ NULL        │
│ chinook │ main    │ PlaylistTrack │ BASE TABLE │ NULL        │
│ chinook │ main    │ Playlist      │ BASE TABLE │ NULL        │
│ chinook │ main    │ todos         │ BASE TABLE │ NULL        │
│ chinook │ main    │ latin         │ BASE TABLE │ NULL        │
│ chinook │ main    │ MediaType     │ BASE TABLE │ NULL        │
│ chinook │ main    │ Track         │ BASE TABLE │ NULL        │
│ chinook │ main    │ Employee      │ BASE TABLE │ NULL        │
│ chinook │ main    │ InvoiceLine   │ BASE TABLE │ NULL        │
│ chinook │ main    │ Customer      │ BASE TABLE │ NULL        │
│ chinook │ main    │ Gen

The functions bellow add some utilities that are useful for working with tables and views in a database.

In [ ]:
#| export

@patch
def datamodel(self: DuckDBPyConnection, table_name:str) ->List[Dict]:
    ''' Returns the data model of a table or view. 
    The columns names, types, nullable status, default value and
    primary key status.'''
    
    return [{'name': r[1], 'type': r[2], 'nullable': not r[3], 'default': r[4], 'pk': r[5]} 
            for r in self.sql(f"PRAGMA table_info='{table_name}'").fetchall()]

In [ ]:
db.datamodel('Artist')


[{'name': 'ArtistId',
  'type': 'INTEGER',
  'nullable': False,
  'default': None,
  'pk': True},
 {'name': 'Name',
  'type': 'VARCHAR',
  'nullable': True,
  'default': None,
  'pk': False}]

In [ ]:
#| export

from dataclasses import field, make_dataclass
def convertTypes(s:str)->type:
    ''' Convert DuckDB types to Python and Numpy types'''
    d = {
        # Built-in types
        'BOOLEAN': bool,
        'BLOB': bytearray,  # For bytes, bytearray can be used in Python
        'DOUBLE': float,
        'BIGINT': int,
        'VARCHAR': str,
        'VARCHAR[]': str,
    
        # NumPy DTypes
        'FLOAT': np.float32,
        'DOUBLE': float,
        'SMALLINT': np.int16,
        'INTEGER': np.int32,
        'TINYINT': np.int8,
        'USMALLINT': np.uint16,
        'UINTEGER': np.uint32,
        'UBIGINT': np.uint64,
        'UTINYINT': np.uint8
    }
    if s in d: return d[s]
    if s[:7]=='DECIMAL': return float
    raise ValueError(f'Unknown type {s}')


import re, keyword
def clean(s):
    s = re.sub(r'\W|^(?=\d)', '_', s)
    return s + '_' if keyword.iskeyword(s) else s

@patch
def dataclass(self: DuckDBPyConnection, 
              table_name:str, # table or view name
              pref='', # prefix to add to the field names
              suf='', # suffix to add to the field names
              cls_name:str = None # defaults to table_name
              ) -> type:
    '''Creates a `dataclass` type from a table or view in the database.'''
    cls_name = cls_name or table_name
    fields = self.datamodel(table_name)
    fields = [(clean(pref+f['name']+suf), convertTypes(f['type']) if not f['nullable'] else convertTypes(f['type'])|None , field(default=f['default'])) for f in fields]
    return make_dataclass(table_name, fields)

In [ ]:
artist_dc = db.dataclass('Artist')
# src = dataclass_src(artist_dc)
# hl_md(src, 'python') # fix error in nbdev_prepare

```python
@dataclass
class Artist:
    ArtistId: int32 = None
    Name: str | None = None

```

In [ ]:
acdc = db.sql(f"select * from artist where artist.Name like 'AC/%'").df().to_dict(orient='records')
acdc

[{'ArtistId': 1, 'Name': 'AC/DC'}]

In [ ]:
acdc_object = artist_dc(**acdc[0])
acdc_object

Artist(ArtistId=1, Name='AC/DC')

### Autocomplete

We want an easy access to table information.

like db`.tables` should work similar to the `.table` cli command.

In [ ]:
#| export

def noop(*args, **kwargs): return None
def identity(x): return x


class _Getter: 
    """ A Getter utility check https://github.com/AnswerDotAI/fastlite """
    def __init__(self, name:str='', type:str='', dir:List=[], get=noop): store_attr()    
    def __dir__(self): return self.dir
    def __str__(self): return ", ".join(dir(self))
    def __repr__(self): return f"{self.type}s: {str(self)}"
    def __contains__(self, s:str): return s in dir(self)
    def __getitem__(self, k): return self.get(k)
    def __getattr__(self, k):
        if k[0]!='_': return self.get(k)
        else: raise AttributeError 

@patch(as_prop=True) # tables
def t(self:DuckDBPyConnection): 
    '''Autocomplete functonality for tables'''
    return _Getter('Tables', 'Table', [r[0] for r in self.base_tables.select('name').fetchall()], self.table)
@patch(as_prop=True) # views
def v(self:DuckDBPyConnection): 
    '''Autocomplete functonality for views'''
    return _Getter('Views', 'View', [r[0] for r in self.views.select('name').fetchall()], self.table)
@patch(as_prop=True) # functions
def fns(self:DuckDBPyConnection): raise NotImplementedError
# def fns(self:DuckDBPyConnection): return _Getter(self, f"SELECT function_name FROM duckdb_functions() WHERE schema_name = '{self.schema}' and internal = False")

@patch(as_prop=True) # secrets
def shh(self:DuckDBPyConnection): raise NotImplementedError
# def shh(self:DuckDBPyConnection): return _Getter(self, f"SELECT name FROM duckdb_secrets()")

@patch
def __repr__(self:DuckDBPyConnection): return f'{self.__class__.__name__} ({self.catalog}::{self.schema})'


In [ ]:
@patch
def _select(self:DuckDBPyRelation, k) -> DuckDBPyRelation:
    return self.select(k) if isinstance(k, str) else self.select(*k)

@patch(as_prop=True)
def c(self:DuckDBPyRelation): 
    '''Column autocomplete'''
    return _Getter('Columns', 'Column', self.columns, self._select)

::: 
 
![Autocomplete in Jupyter](images/autocomplete.png){.lightbox}

:::

In [ ]:

a = db.t.Album.c['AlbumId', 'Title'].limit(4)
a

┌─────────┬───────────────────────────────────────┐
│ AlbumId │                 Title                 │
│  int32  │                varchar                │
├─────────┼───────────────────────────────────────┤
│       1 │ For Those About To Rock We Salute You │
│       2 │ Balls to the Wall                     │
│       3 │ Restless and Wild                     │
│       4 │ Let There Be Rock                     │
└─────────┴───────────────────────────────────────┘

### Relation utilities

Once we know that a certain `DuckDBPyRelation` is a table (or view), we can also make it keep some valuable props.

In [ ]:
#| export

_saved = {}

@patch
def _set(self:DuckDBPyRelation, k, v):
    global _saved
    # use hash to avoid clashes
    _saved[str(hash(self))+'_'+k] = v

@patch
def _get(self:DuckDBPyRelation, key):
    global _saved
    k = str(hash(self))+'_'+key
    return _saved[k] if k in _saved else None

def custom_dir(c, add): return dir(type(c)) + list(c.__dict__.keys()) + add

def create_patch_property(name):
    @patch(as_prop=True)
    def prop(self: DuckDBPyRelation):
        return self._get(name)
    return prop

props = ['cls', 'rel', 'model', 'meta']
for p in props: setattr(DuckDBPyRelation, p, create_patch_property(p))

@patch
def __dir__(self:DuckDBPyRelation): return custom_dir(self, props)
    
def create_prop(c, name, f): setattr(c, name, property(f))
@patch(as_prop=True)
def cls(self:DuckDBPyRelation): return self._get('cls')

@patch(as_prop=True)
def model(self:DuckDBPyRelation): return self._get('model')

@patch(as_prop=True)
def meta(self:DuckDBPyRelation): return self._get('meta')

@patch(as_prop=True)
def rel(self:DuckDBPyRelation): return self._get('rel')


@patch
def table(self:DuckDBPyConnection, name:str, schema:str= None) -> DuckDBPyRelation:
    if isinstance(name, Union[List, Set, Tuple]): return [self.table(n) for n in name]
    if not isinstance(name,str): raise AttributeError
    r = self.tables.filter(f"name == '{name}' and schema == '{schema or self.schema}'")
    catalog, schema, name, type, comment = r.fetchone()
    tbl = self.sql(f"from {schema}.{name}")
    tbl = tbl.set_alias(f"{schema}.{name}")
    tbl._set('cls', self.dataclass(name))
    tbl._set('model', self.datamodel(name))
    meta = {'base': self, 'catalog': catalog, 'schema': schema, 'name': name, 'type': type, 'comment': comment, 'shape': tbl.shape}
    tbl._set('meta', meta)
    tbl._set('rel', tbl)
    return tbl

Let's also improve the representation of `Relations`.

In [ ]:

@patch
def __str__(self:DuckDBPyRelation): return f'{self.alias}'

@patch
def __repr__(self:DuckDBPyRelation): 
    return f"<{self.__class__.__name__} {self.meta['type'] if self.meta else ''} :: _{self.alias}_ ({self.shape[0]} rows, {self.shape[1]} cols)>"

@patch
def _repr_markdown_(self: DuckDBPyRelation): 
    markdown =  self.__repr__()+"\n\n"
    if self.meta and self.meta['comment']: markdown += f"> {self.meta['comment']}\n\n"
    df = self.df()
    if self.shape[0] > 6: 
        head = df.head(3)
        tail = df.tail(3)
        ellipsis = pd.DataFrame([["..."] * df.shape[1]], columns=df.columns)
        df = pd.concat([head, ellipsis, tail])
    markdown += f"{df.to_markdown(index=False, headers="keys", tablefmt="pipe" )}\n\n"
    return markdown


In [ ]:
db.sql('select * from Album')

<DuckDBPyRelation  :: _unnamed_relation_2ca56d641557b4dc_ (347 rows, 3 cols)>

| AlbumId   | Title                                              | ArtistId   |
|:----------|:---------------------------------------------------|:-----------|
| 1         | For Those About To Rock We Salute You              | 1          |
| 2         | Balls to the Wall                                  | 2          |
| 3         | Restless and Wild                                  | 2          |
| ...       | ...                                                | ...        |
| 345       | Monteverdi: L'Orfeo                                | 273        |
| 346       | Mozart: Chamber Music                              | 274        |
| 347       | Koyaanisqatsi (Soundtrack from the Motion Picture) | 275        |



In [ ]:
db.t.Genre

<DuckDBPyRelation BASE TABLE :: _main.Genre_ (25 rows, 2 cols)>

| GenreId   | Name        |
|:----------|:------------|
| 1         | Rock        |
| 2         | Jazz        |
| 3         | Metal       |
| ...       | ...         |
| 23        | Alternative |
| 24        | Classical   |
| 25        | Opera       |



In [ ]:
a

<DuckDBPyRelation  :: _Album_ (4 rows, 2 cols)>

|   AlbumId | Title                                 |
|----------:|:--------------------------------------|
|         1 | For Those About To Rock We Salute You |
|         2 | Balls to the Wall                     |
|         3 | Restless and Wild                     |
|         4 | Let There Be Rock                     |



### Replacement Scans

You may be asking yourself why I am patching `DuckDBPyRelation` and `DuckDBPyConnection` instead of subclassing them.
The problem is that these classes do not allow subclassing.  They do not implement `__init__`.

We could have create our own classes like `Database` and `Table` and just wrap DuckDBPy objects. But then we would loose a very nice feature of the PyRelation objects.....


*replacement scans*.


In [ ]:
a = db.t.Album

In [ ]:
db.sql("select * from a")

<DuckDBPyRelation  :: _unnamed_relation_e2365f83b82bfa49_ (347 rows, 3 cols)>

| AlbumId   | Title                                              | ArtistId   |
|:----------|:---------------------------------------------------|:-----------|
| 1         | For Those About To Rock We Salute You              | 1          |
| 2         | Balls to the Wall                                  | 2          |
| 3         | Restless and Wild                                  | 2          |
| ...       | ...                                                | ...        |
| 345       | Monteverdi: L'Orfeo                                | 273        |
| 346       | Mozart: Chamber Music                              | 274        |
| 347       | Koyaanisqatsi (Soundtrack from the Motion Picture) | 275        |



I did not had to use a f-string and pass the variable.  DuckDBPy objects (as well as Pandas and Polars Dataframes, Arrow tables, and Datasets) are replaced in the query automagically.

In [ ]:
db.sql(f"select * from {a}")

<DuckDBPyRelation  :: _unnamed_relation_eb016f018b76cdf0_ (347 rows, 3 cols)>

| AlbumId   | Title                                              | ArtistId   |
|:----------|:---------------------------------------------------|:-----------|
| 1         | For Those About To Rock We Salute You              | 1          |
| 2         | Balls to the Wall                                  | 2          |
| 3         | Restless and Wild                                  | 2          |
| ...       | ...                                                | ...        |
| 345       | Monteverdi: L'Orfeo                                | 273        |
| 346       | Mozart: Chamber Music                              | 274        |
| 347       | Koyaanisqatsi (Soundtrack from the Motion Picture) | 275        |



In [ ]:
str(a)

'main.Album'

### Improving Database import

In [ ]:
from pathlib import Path
from typing import Optional, Literal



@patch
def __contains__(self:DuckDBPyConnection, name:str):
    return name in [r[0] for r in self.tables.select('name').fetchall()]

@patch
def drop(self:DuckDBPyConnection, table_name:str):
    '''Drop a table or view'''
    self.tables.filter("name == 'a_view'")
    if table_name not in self: raise ValueError(f"Table {table_name} does not exist")
    self.sql(f"DROP TABLE {table_name}")
    
@patch
def _create(self: DuckDBPyConnection, 
            type: str, fileglob: str, 
            table_name: Optional[str] = None, 
            filetype: Optional[Literal['csv', 'xlsx', 'json', 'parquet']] = None, 
            replace: bool = False, 
            *args, **kwargs):
  
    filepath = Path(fileglob)
    name = table_name or filepath.stem
    if name in self and not replace: raise ValueError(f"Table {name} already exists")
    if name in self: self.drop(name)
    
    filetype = filetype or filepath.suffix[1:]
    if filetype == 'xlsx':
        self.install_extension('spatial')
        self.load_extension('spatial') # for excel import/export
        fn = 'st_read'
        options = ', '.join(f"{key}={repr(value)}" for key, value in kwargs.items())
        self.sql(f"CREATE {type} {name} AS SELECT * FROM {fn}('{str(filepath)}' {options})")
    else:
        (getattr(self, f'read_{filetype}')(fileglob, *args, **kwargs)).to_table(name)


@patch
def create_table(self: DuckDBPyConnection, 
                 fileglob: str, # file path or glob
                 table_name: Optional[str] = None, # table name
                 filetype: Optional[Literal['csv', 'xlsx', 'json', 'parquet', 'sqlite']] = None, # file type
                 replace: bool = False, # replace existing table
                 *args, **kwargs 
                 ):
    '''Create a table from a file'''
    return self._create('TABLE', fileglob, table_name, filetype, replace, *args, **kwargs)

@patch
def create_view(self: DuckDBPyConnection, 
                 fileglob: str, # file path or glob
                 view_name: Optional[str] = None, # view name
                 filetype: Optional[Literal['csv', 'xlsx', 'json', 'parquet', 'sqlite']] = None, # file type
                 replace: bool = False,  # replace existing view
                 *args, **kwargs
                 ):
    '''Create a view from a file'''
    return self._create('VIEW', fileglob, view_name, filetype, replace, *args, **kwargs)
    

In [ ]:
db.create_table('../data/username.latin1.csv', 'latin', replace=True)
db.tables

<DuckDBPyRelation  :: _unnamed_relation_3eb4ad76c1db552e_ (15 rows, 5 cols)>

| catalog   | schema   | name        | type       | comment   |
|:----------|:---------|:------------|:-----------|:----------|
| chinook   | main     | Invoice     | BASE TABLE |           |
| chinook   | main     | latin1      | VIEW       |           |
| chinook   | main     | Playlist    | BASE TABLE |           |
| ...       | ...      | ...         | ...        | ...       |
| chinook   | main     | InvoiceLine | BASE TABLE |           |
| chinook   | main     | Customer    | BASE TABLE |           |
| chinook   | main     | Genre       | BASE TABLE |           |



In [ ]:
db.tables
#  db.sql("select distinct database_name, schema_name, table_name, column_name from duckdb_columns()")
# db.sql("attach database '../data/chinook.sqlite' as sqlite")

<DuckDBPyRelation  :: _unnamed_relation_0555d0f4c1ea15b1_ (15 rows, 5 cols)>

| catalog   | schema   | name       | type       | comment     |
|:----------|:---------|:-----------|:-----------|:------------|
| chinook   | main     | Playlist   | BASE TABLE |             |
| chinook   | main     | Album      | BASE TABLE | Album table |
| chinook   | main     | Artist     | BASE TABLE |             |
| ...       | ...      | ...        | ...        | ...         |
| chinook   | main     | Genre      | BASE TABLE |             |
| chinook   | main     | todos      | BASE TABLE |             |
| chinook   | main     | latin1view | BASE TABLE |             |



In [ ]:
db.create_view('../data/username.latin1.csv', 'latin1view', replace=True)

In [ ]:
# from fastcore.test import test_fail
# test_fail(db.drop, 'banana') # fix error in nbdev_prepare

In [ ]:
db.create_table('https://jsonplaceholder.typicode.com/todos/', 'todos', filetype='json', replace=True)

In [ ]:
db.t.todos.limit(10)

<DuckDBPyRelation  :: _main.todos_ (10 rows, 4 cols)>

| userId   | id   | title                                        | completed   |
|:---------|:-----|:---------------------------------------------|:------------|
| 1        | 1    | delectus aut autem                           | False       |
| 1        | 2    | quis ut nam facilis et officia qui           | False       |
| 1        | 3    | fugiat veniam minus                          | False       |
| ...      | ...  | ...                                          | ...         |
| 1        | 8    | quo adipisci enim quam ut ab                 | True        |
| 1        | 9    | molestiae perspiciatis ipsa                  | False       |
| 1        | 10   | illo est ratione doloremque quia maiores aut | True        |



In [ ]:
db.create_table('https://huggingface.co/datasets/ibm/duorc/resolve/refs%2Fconvert%2Fparquet/ParaphraseRC/test/0000.parquet', 'hf_movies')

In [ ]:
db.t.hf_movies.limit(2)

<DuckDBPyRelation  :: _main.hf_movies_ (2 rows, 7 cols)>

| plot_id    | plot                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | title                 | question_id                          | question                      | answers           | no_answer   |
|:-----------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------|:-------------------------------------|:------------------------------|:------------------|:------------|
| /m/0278wws | This section needs expansion. You can help by adding to it. (October 2014)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | The Last Horror Movie | 67386318-d155-189c-9569-e35bf46a338c | Who portrays Max Parry?       | ['Kevin Howarth'] | False       |
|            | The film follows Max Parry (Kevin Howarth), a disturbed wedding video cameraman, and his unnamed assistant (Mark Stevenson) as they perform several murders that they have videotaped. The two have used a video store tape in order to record the proceedings, breaking the fourth wall and insinuating that the copy of the film being watched is the only existing version of the tape. Throughout the film Max uses meta-references in order to show off his gruesome activities as a serial killer. The film raises questions surrounding visceral pleasure, this can be seen in one scene in particular during which the audience cannot see the victims (two at once) being murdered, Max Parry then asks the audience "I bet you wanted to see that, and if you didn't, why are you still watching?" |                       |                                      |                               |                   |             |
|            | At the end of the film the audience is left to believe that since they are watching the only copy of the film, that they will potentially become one of Max's victims.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |                       |                                      |                               |                   |             |
| /m/0278wws | This section needs expansion. You can help by adding to it. (October 2014)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | The Last Horror Movie | 7255bfe4-67b0-7a6d-22ee-1245e4221f3f | Where does Max murder people? | []                | True        |
|            | The film follows Max Parry (Kevin Howarth), a disturbed wedding video cameraman, and his unnamed assistant (Mark Stevenson) as they perform several murders that they have videotaped. The two have used a video store tape in order to record the proceedings, breaking the fourth wall and insinuating that the copy of the film being watched is the only existing version of the tape. Throughout the film Max uses meta-references in order to show off his gruesome activities as a serial killer. The film raises questions surrounding visceral pleasure, this can be seen in one scene in particular during which the audience cannot see the victims (two at once) being murdered, Max Parry then asks the audience "I bet you wanted to see that, and if you didn't, why are you still watching?" |                       |                                      |                               |                   |             |
|            | At the end of the film the audience is left to believe that since they are watching the only copy of the film, that they will potentially become one of Max's victims.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |                       |                                      |                               |                   |             |



In [ ]:
db.drop('hf_movies')


In [ ]:

db.create_table('../data/example.xlsx')

In [ ]:
db.drop('example')

#### Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()